# Primera entrega
Luis Montenegro - 21699<br>
Javier Prado - 21486<br>
Bryan España<br>
Ángel Herrarte<br>


## Limpieza de datos
En esta entrega se hará una recolección y limpieza de datos solamente. No habrá modelado ni análisis. 
<br>Solamente velar por la integridad, coherencia y cohesión del conjunto de datos.facilidad de manejo.

In [1]:
# !pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from collections import defaultdict
import unicodedata
import re

### Conversión de .xls a .csv

Debido a que los archivos crudos descargados desde la página del Mineduc vienen en formato .xls formateado como .html tenemos que hacer cambio de eso.<br>
Esto por múltiples razones:
- Mejorar la estructura de los datos.
- Preservar solamente la información requerida (los .xls almaceban más páginas e información irrelevante)
- Mayor falicidad de usar y obtener los datos si están en formato delimitado por comas.
- Tener datos más ligeros ya que no acarreamos con mucha información innecesaria. 

In [3]:
def transform_html_to_csv(input_path: str, output_path: str=None, tables_to_convert: list[int] = [9]) -> None:
    '''
    Turns .html files into .csv files

    Params:
        input_path: where the .html files are stored
        output_path: where the .csv will be stored
    Returns:
        out: None
    '''
    if output_path is None:
        output_path = input_path 
    else:
        os.makedirs(output_path, exist_ok=True)

    for filename in tqdm(os.listdir(input_path), desc='Converting Files'):
        if filename.endswith('.xls'):
            html_path:str = os.path.join(input_path, filename)
            base_name:str = os.path.splitext(filename)[0]
            # try to read the excel and copy it into a .csv file
            try: 
                tables:list[pd.DataFrame] = pd.read_html(html_path)
                # if no tables, skip
                if not tables:
                    print(f"No tables found in '{filename}'")
                    continue

                # convert each of the tables selected to .csv
                for i, df in enumerate(tables):
                    if i in tables_to_convert:
                        csv_filename = f"{base_name}_table{i}.csv"
                        csv_path = os.path.join(output_path, csv_filename)
                        df.to_csv(csv_path, index=False)
                        print(f"Converted '{filename}' to '{csv_filename}'") # print all of the transformations of the .xls to its respective .csv files
                    
            except Exception as e:
                print(f"Error processing file '{filename}' : {e}")

def remove_files(path: str, extension: str) -> None:
    '''
    Removes all files that have a certain extension
    Input:
        path: folder path where .xls to be removed are stored
    '''
    file_list: list[str] = os.listdir(path)
    if not file_list:
        print(f"No files removed since there was none found: '{path}'")
        return

    for filename in file_list:
    
        if filename.endswith(extension):
            try:
                xls_path = os.path.join(path, filename)
                os.remove(xls_path)
            except Exception as e:
                print(f"Error removing file '{filename}' : {e}")


In [4]:
# start by turning .xls files to .csv files
transform_html_to_csv(input_path="../Dataset_raw", output_path="../Dataset_cleaned", tables_to_convert=[9])

FileNotFoundError: [Errno 2] No such file or directory: '../Dataset_raw'

In [ ]:
# remove .xls files if necessary
remove_files(path="../Dataset_raw", extension='.xls')

### Estandarizar la estructura de los .csv
Es importante que los .csv con los que estamos trabajando tengan una misma estructura. Por lo tanto, debemos asegurarnos que todos sean iguales. <br>
Esto lo hacemos con la finalidad de evitar errores al limpiar los datos más adelante o al concatenarlos al final. 

Queremos que todos los .csv tengan 17 títulos de columnas en orden tipo:<br>
CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL<br>

Otra cosa a considerar es eliminar la primera fila y reajustar índices. Debido a que la primera fila son puros números en vez de los nombres de las columnas. Al igual que debemos
eliminar la última fila que es todos nulos. 

Para los que tenían error de formato o formato diferente solamente copiamos los datos en un .csv ya que usualmente eso se debe a que solamente había 1 establecimiento encontrado.


In [ ]:
COLUMNS = ["CODIGO","DISTRITO","DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION","TELEFONO","SUPERVISOR","DIRECTOR","NIVEL","SECTOR","AREA","STATUS","MODALIDAD","JORNADA","PLAN","DEPARTAMENTAL"]

def clean_headers_and_trailers(path: str) -> None:
    '''
    Removes first row and reindexes the data within.
    Also removes the last row that contains only nulls
    '''
    for file in tqdm(os.listdir(path), desc="Converting files"):
        if file.endswith('.csv'):
            try:
                f_path = os.path.join(path, file) # path to the file
                df: pd.DataFrame = pd.read_csv(f_path)
                
                # drop first row and reindex if the first row are digits and not columns
                if all(col.strip().isdigit() for col in df.columns):
                    df.columns = df.iloc[0]
                    df = df[1:].reset_index(drop=True)

                # remove last row if only nulls are found
                if df.tail(1).isnull().all(axis=1).iloc[0]:
                    df = df.iloc[:-1]
                    print(f"Removed last row: {file}")
                
                df.to_csv(f_path, index=False)  
                
                print(f"Finished: {file}")
            
            except Exception as e:
                print(f"Error processing '{file}': {e}")


In [ ]:
# clean headers and trailers
# clean_headers_and_trailers(path = "../Dataset_cleaned")

### Plan para limpieza de datos
Posteriormente a la transformación de formato y filtración a solo datos crudos de nuestro interés, procederemos a limpiar los datos en sí. <br>
1. Verificaremos si existe datos NA o Null en los archivos. Llenaremos los datos faltantes con datos artificiales ya que no podemos eliminar ningún registro <br>
debido a que si eliminamos alguno, sería una sede faltante. Cosa que afectaría de gran manera el conjunto de datos.
2. Homogenizaremos la información. Es decir, que todos los archivos tengan el mismo formato i.e. todos los nombres estén escritos igual, los apellidos que tienen tildes se escriban igual en cada archivo,
que las palabras vengan o solo en mayúsculas, solo en minúsculas, etc.
3. Identificaremos las columnas que más trabajo de reajuste necesiten.
- Nombres (Supervisor, Director, Establecimiento, Sector): debido a que puede que exista apellidos iguales pero escritos distinto y eso genere problemas a la hora de verlos. Digamos, puede que Hernández aparezca con tilde en unos registros y en otros no. También es bueno verificar que estén escritos todos o en mayúsculas o en minúsculas (o todos iguales), ya que digamos, si tenemos Santa rosa en un registro y Santa Rosa en otro, a la hora de hacer un encoding estos resultarán con dos valores distintos. 
- Dirección: Asegurarnos que las direcciones lleven una estructura similar.
- Códigos: Verificar que los códigos de los registros sean únicos y evitar tener repetidos.



El conjunto de datos corresponde a los establecimientos educativos de Guatemala que llegan hasta el nivel diversificado. Está organizado en 85 archivos CSV, uno por cada departamento y municipio del país.

Cada archivo contiene información detallada de cada establecimiento, como el nombre del centro, su ubicación, datos de contacto, modalidad de enseñanza y otros datos administrativos.


Los archivos csv estan clasificados por departamentos de Guatemala. Las variables que contiene esta dataset son las siguientes sumando un total de 17 variables a analizar.

- CODIGO: código único del establecimiento

- DISTRITO: distrito educativo

- DEPARTAMENTO: nombre del departamento

- MUNICIPIO: municipio donde se ubica

- ESTABLECIMIENTO: nombre del centro educativo

- DIRECCION: dirección física

- TELEFONO: número de contacto

- SUPERVISOR: nombre del supervisor

- DIRECTOR: nombre del director

- NIVEL: nivel educativo (ej. Básico, Diversificado)

- SECTOR: sector oficial o privado

- AREA: área urbana o rural

- STATUS: estado del centro (ej. Abierta)

- MODALIDAD: modalidad lingüística (ej. Monolingüe, Bilingüe)

- JORNADA: jornada de estudio (ej. Matutina, Vespertina)

- PLAN: plan educativo (ej. Diario, Fin de semana)

- DEPARTAMENTAL: nombre del departamento de adscripción

### Encontrar Null o NA en cualesquiera archivos
Comenzaremos viendo si existe cualesquiera archivos con datos faltantes.

In [2]:
dataset_path: str = "../Dataset_cleaned" # define folder path as a variable for easier handling
report_path: str = "../Data_null_report"

def get_null_stats(path:str) -> None:
        '''
        Prints the columns with most null counts and the files with most nulls
        '''
        df: pd.DataFrame = pd.read_csv(path)
        print("\n Top columns with the most nulls:")
        top_columns = df.groupby('column')['nulls'].sum().sort_values(ascending=False).head(10)
        print(top_columns)

        print("\n Files with the most nulls:")
        top_files = df.groupby('file_name')['nulls'].sum().sort_values(ascending=False).head(10)
        print(top_files)

def count_null_instances(path:str, report_name:str = "null_report.csv") -> None:
    '''
    Counts how many missing instances are found per column per .csv and saves it into a .csv
    '''
    files: list[str] = os.listdir(path=path)
    report_file: str = os.path.join(report_path, report_name)
    os.makedirs(name=report_path, exist_ok=True) # make a new directory to save this information
    data:list[dict] = []
    for file in tqdm(files, desc="Counting nulls.."):
        file_path:str = os.path.join(path, file)
        try:
            df: pd.DataFrame = pd.read_csv(file_path) 
            null_counts = df.isnull().sum() # register nulls found in the df per column
            for col, count in null_counts.items():
                data.append({
                    "file_name": file,
                    "column": col,
                    "nulls": count
                })
        
        except Exception as e:
            print(f"Error processing {file} : {e}")
    
    report_df: pd.DataFrame = pd.DataFrame(data=data)
    report_df.to_csv(report_file, index=False) # not necessary to save it as a .csv



In [3]:
count_null_instances(path=dataset_path) # count nulls found per column in each .csv

Counting nulls..:  28%|██▊       | 24/85 [00:00<00:00, 236.20it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Counting nulls..: 100%|██████████| 85/85 [00:00<00:00, 318.51it/s]


In [4]:
get_null_stats(path="../Data_null_report/null_report.csv")


 Top columns with the most nulls:
column
TELEFONO      4320
DIRECTOR       116
DIRECCION       23
MUNICIPIO        0
SUPERVISOR       0
STATUS           0
SECTOR           0
PLAN             0
NIVEL            0
AREA             0
Name: nulls, dtype: int64

 Files with the most nulls:
file_name
alta_verapaz_primaria_table9.csv        633
alta_verapaz_preprimaria_table9.csv     376
quiche_primaria_table9.csv              334
peten_primaria_table9.csv               311
jutiapa_primaria_table9.csv             277
huehuetenango_primaria_table9.csv       258
santa_rosa_primaria_table9.csv          225
san_marcos_primaria_table9.csv          225
baja_verapaz_primaria_table9.csv        166
huehuetenango_preprimaria_table9.csv    138
Name: nulls, dtype: int64


Como podemos observar, las columnas con mayor recuento de nulos son de TELEFONO, DIRECTOR y DIRECCION. Mientras que los archivos con más nulos son altaverapaz con primaria y preprimaria. 
Por lo tanto, lo que debemos hacer con esto es darles un valor a los nulos. Por ejemplo, un número predeterminado cuando no hay, o un texto único para cuando no haya nombres o direciones, etc. <br>
De esta manera no eliminamos ni afectamos el conjunto de datos y aún logramos clasificarlos dentro de su propia categoría.

### Verificación de tipos de datos y consistencia entre archivos
Antes de comenzar a llenar los campos vacíos, debemos asegurarnos que los campos que estamos llenando contengan los mismos tipos. <br>
Es decir, que no tengamos archivos .csv que tengan tipos int64 y otros object bajo la misma columna. Ya que esto generará problemas a la hora<br>
de trabajar los datos y puede generar dificultades de manejo.


In [5]:
# check datatypes for each column within a file
def check_datatypes(file: str, examples: bool = False) -> tuple[defaultdict[set], defaultdict[dict]]:
    df: pd.DataFrame = pd.read_csv(file)
    type_map = defaultdict(set)
    examples_map = defaultdict(dict)
    for col in df.columns:
        dtype = str(df[col].dtype)
        type_map[col].add(dtype)
        
        if examples:
            for val in df[col]:
                if pd.notnull(val):
                    val_type = str(pd.Series([val]).dtype)
                    if val not in examples_map[col]:
                        examples_map[col][val_type] = val
                    break
    return type_map, examples_map

def check_datatype_differences(path: str, examples: bool = False) -> None:
    '''
    Checks if there is any difference between columns' datatypes across all files
    '''
    
    global_type_map = defaultdict(set)
    global_examples_map = defaultdict(dict)
    for file in os.listdir(path):
        if file.endswith('.csv'):
            try: 
                f_path: str = os.path.join(path, file)
                type_map, examples_map = check_datatypes(f_path, examples=examples)
                for col, types in type_map.items():
                    global_type_map[col].update(types)
                
                if examples:
                    for col, example_dict in examples_map.items():
                        for dtype, val in example_dict.items():
                            if dtype not in global_examples_map[col]:
                                global_examples_map[col][dtype] = val

            except Exception as e:
                print(f"Unexpected error on file '{file}': {e}")

    print(f"Columns and their types found")
    for col, types in global_type_map.items():
        print(f"- {col} : {types}")
        if examples and len(types) > 1:
            for dtype, value in global_examples_map[col].items():
                print(f"{dtype} : {repr(value)}")

In [6]:
check_datatype_differences(path=dataset_path, examples=True)

Columns and their types found
- CODIGO : {'object'}
- DISTRITO : {'object'}
- DEPARTAMENTO : {'object'}
- MUNICIPIO : {'object'}
- ESTABLECIMIENTO : {'object'}
- DIRECCION : {'object'}
- TELEFONO : {'int64', 'float64', 'object'}
float64 : 79450881.0
object : '77661038'
int64 : 79414031
- SUPERVISOR : {'object'}
- DIRECTOR : {'object'}
- NIVEL : {'object'}
- SECTOR : {'object'}
- AREA : {'object'}
- STATUS : {'object'}
- MODALIDAD : {'object'}
- JORNADA : {'object'}
- PLAN : {'object'}
- DEPARTAMENTAL : {'object'}


Como podemos observar arriba, tenemos que los tipos encontrados son en su mayoría object, pero la columna TELEFONO posee tanto object como int64 y float 64. <br>
Eso es un problema debido a que no podemos manejar esos 3 tipos a lo largo y ancho del conjunto de datos. Por lo que será necesario indagar a fondo de por qué aparece
esos tipos en el área de teléfono. <br>
De igual forma será necesario cambiar los tipo object para que todos sean un tipo concreto y no ambiguo. Por ejemplo, darles tipo int64 si son meramente numéricos como los <br>
números telefónicos o string si son direcciones o todo lo demás. 

In [7]:
import re

def clean_string(value: str) -> str:
    if pd.isnull(value):
        return value
    else:
        value = str(value).strip()
        value = value.replace('""', '"')
        return value


def change_dataframe_types_structure(file_path:str, columns: list[str] = None) -> pd.DataFrame:
    '''
    Creates a dataframe and changes all of the types to string so 
    it returns a uniform dataframe without type variance and without unnecesary quotes, punctuation marks and others
    '''
    df: pd.DataFrame = pd.read_csv(file_path)
    df = df.astype({col: 'string' for col in df.select_dtypes(include=['object', 'int64', 'float64']).columns})

    if columns is None:
        columns = df.columns
    for col in columns:
        if col in df.columns:
            try:
                df[col] = df[col].map(clean_string)
            except KeyError as k:
                print(f"No column found: {k}")
            except Exception as e:
                print(f"Unexpected error on conversion: {e}")
    
    return df


La función descrita arriba la utilizaremos para cambiar los tipos en los dataframes para ejecutar los cambios en los dataframe bajo los mismos tipos. 

### Asignar valores a los datos inexistentes o nulos
Ahora lo que haremos, como se dijo previamente, es asignarle valores a las celdas nulas. <br>
Los siguientes valores predeterminados para las celdas nulas serán los siguientes:<br>

DIRECCION : DESCONOCIDO<br>
TELEFONO : 00000000<br>
DIRECTOR : DESCONOCIDO<br>

Pero primero observaremos los tipos de cada columna para tener mejor definición de qué hay en cada una y que no exista<br>
discrepancia entre los tipos a lo largo de las columnas y archivos.

In [8]:
# fill missing data with predetermined values
missing_data_dict: dict = {"DIRECCION":"DESCONOCIDO", "TELEFONO": "00000000", "DIRECTOR": "DESCONOCIDO"} 


def fill_nulls_with_values(path:str, value_dict: dict, columns_to_clean: list[str] = None) -> list[pd.DataFrame]:
    '''
    Changes null instances within columns and fills them with predetermined values
    '''
    df_list: list[pd.DataFrame] = []
    for file in os.listdir(path):
        if file.endswith('.csv'):
            f_path: str = os.path.join(path, file)
            df: pd.DataFrame = change_dataframe_types_structure(f_path, columns=columns_to_clean) # returns a df that has all datatypes as strings and formats them correctly 
            try:
                for col_name, default_val in value_dict.items():
                    df[col_name] = df[col_name].fillna(default_val)
                
            except Exception as e:
                print(f"Error while filling nulls with '{file}' : {e}")
            finally:
                df_list.append(df)
    
    return df_list

            

In [9]:
df_list: list[pd.DataFrame] = fill_nulls_with_values(path=dataset_path, value_dict=missing_data_dict, columns_to_clean=['ESTABLECIMIENTO'])
df_list[0].iloc[:9, :9]

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR
0,02-01-0027-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO TECNICO INDUSTRIAL MIXTO GUASTATOYA,BARRIO EL CALVARIO,79450881.0,CARLA MARLENY ALDANA RODAS,JOSÉ ARTURO LÓPEZ ORTIZ
1,02-01-0028-46,02-014,EL PROGRESO,GUASTATOYA,COLEGIO DE CIENCIAS COMERCIALES EL PROGRESO,BARRIO EL PORVENIR,79451265.0,CARLA MARLENY ALDANA RODAS,DANIEL LÓPEZ SOLÍS
2,02-01-0031-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,BARRIO EL PORVENIR,54422753.0,CARLA MARLENY ALDANA RODAS,ILIANA ARACELY LÁZARO HERNÁNDEZ
3,02-01-0045-46,02-014,EL PROGRESO,GUASTATOYA,INSTITUTO DE EDUCACIÓN MEDIA POR MADUREZ,BARRIO EL PORVENIR,56719955.0,CARLA MARLENY ALDANA RODAS,NORA REBECA IBAÑEZ MORÁN
4,02-01-0049-46,02-021,EL PROGRESO,GUASTATOYA,CENTRO MUNICIPAL DE EDUCACIÓN EXTRAESCOLAR -CEEX-,COLONIA LINDA VISTA,58250827.0,MARTA ELIDA CARIAS HERNANDEZ DE GARCIA,SANDRA PAOLA MORALES GARCÍA
5,02-01-0054-46,02-012,EL PROGRESO,GUASTATOYA,COLEGIO EVANGELICO TORRE FUERTE,"COLONIA HICHOS, BARRIO EL PORVENIR",79451993.0,CARLOTA EUGENIA ALBUREZ AGUILAR,IRIS ORFELÍ LOAIZA MOSCOSO
6,02-01-0061-46,02-021,EL PROGRESO,GUASTATOYA,PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA -PR...,BARRIO LAS JOYAS CEMENTERIO VIEJO,79637575.0,MARTA ELIDA CARIAS HERNANDEZ DE GARCIA,MARTA ELIDA CARIAS
7,02-01-0062-46,02-021,EL PROGRESO,GUASTATOYA,CENTRO DE EDUCACIÓN EXTRAESCOLAR -CEEX- DEPART...,"BARRIO LAS JOYAS, CEMENTERIO VIEJO",37962236.0,MARTA ELIDA CARIAS HERNANDEZ DE GARCIA,HELLEN CELESTE MARROQUIN CORADO
8,02-01-0066-46,02-012,EL PROGRESO,GUASTATOYA,COLEGIO MIXTO INTEGRAL EL PROGRESO,"COLONIA HICHOS, BARRIO EL PORVENIR",47406758.0,CARLOTA EUGENIA ALBUREZ AGUILAR,DELFINA MARISOL PAZOS RAMOS


### Transformar las palabras a forma estándar; quitar tildes y diéresis

Una vez que hemos llenado los valores faltantes, es crucial estandarizar el formato del texto en todo el conjunto de datos. Esto es especialmente importante para garantizar consistencia y evitar problemas en análisis posteriores donde palabras iguales pero escritas de forma diferente sean tratadas como distintas.

**Objetivos de esta estandarización:**

1. **Eliminar acentos y caracteres especiales**: Normalizar palabras como "José" → "Jose", "Quiché" → "Quiche"
2. **Unificar formatos por tipo de campo**:
   - **Establecimientos**: TODO EN MAYÚSCULAS para consistencia
   - **Nombres propios** (Director/Supervisor): Formato Título (Primera Letra Mayúscula)
   - **Direcciones**: Expandir abreviaciones (Ave → Avenida, Km → Kilometro)
   - **Teléfonos**: Formato uniforme de 8 dígitos sin separadores
3. **Limpiar espacios y puntuación**: Eliminar espacios múltiples y comillas redundantes

**Transformaciones específicas aplicadas:**

- **Normalización Unicode**: Separar caracteres base de sus acentos y eliminar los acentos
- **Estandarización de direcciones**: "5a. ave 1-23 zona 4" → "5A AVENIDA 1-23 ZONA 4"
- **Formato de teléfonos**: "7794-5104" → "77945104", "79529782.0" → "79529782"
- **Nombres propios**: "JOSÉ MARÍA HERNÁNDEZ" → "Jose Maria Hernandez"
- **Establecimientos**: "Colegio San José" → "COLEGIO SAN JOSE"

Esta estandarización es fundamental para:
- Evitar duplicados por diferencias tipográficas
- Facilitar búsquedas y filtros posteriores  
- Garantizar consistencia en el conjunto de datos final
- Preparar los datos para análisis automatizados

**Proceso:** Se aplicará la estandarización a todos los DataFrames que ya contienen los valores nulos rellenados, manteniendo la integridad de los datos mientras se mejora su formato y consistencia.

In [13]:
def remove_accents_and_special_chars(text: str) -> str:
    """Remueve tildes, diéresis y normaliza caracteres especiales"""
    if pd.isnull(text):
        return text
    
    text = str(text).strip()
    # Normalizar unicode y remover acentos
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    return text

def standardize_phone_number(phone: str) -> str:
    """Estandariza números telefónicos a formato consistente (8 dígitos)"""
    if pd.isnull(phone) or phone == "00000000":
        return "00000000"
    
    phone_str = str(phone).strip()
    digits_only = re.sub(r'\D', '', phone_str)
    
    if len(digits_only) == 8:
        return digits_only
    elif len(digits_only) > 8:
        return digits_only[-8:]
    elif len(digits_only) < 8 and len(digits_only) > 0:
        return digits_only.zfill(8)
    else:
        return "00000000"

def standardize_address(address: str) -> str:
    """Estandariza formato de direcciones"""
    if pd.isnull(address) or address == "DESCONOCIDO":
        return "DESCONOCIDO"
    
    address = str(address).strip().upper()
    
    # Estandarizar abreviaciones comunes
    replacements = {
        r'\bAV\b\.?': 'AVENIDA',
        r'\bAVE\b\.?': 'AVENIDA', 
        r'\bCALL\b\.?': 'CALLE',
        r'\bKM\b\.?': 'KILOMETRO',
        r'\bZON\b\.?': 'ZONA',
        r'\bALD\b\.?': 'ALDEA'
    }
    
    for pattern, replacement in replacements.items():
        address = re.sub(pattern, replacement, address)
    
    address = re.sub(r'\s+', ' ', address)
    return address.strip()

def comprehensive_text_cleaning(text: str, field_type: str = 'general') -> str:
    """Aplica limpieza comprehensiva según el tipo de campo"""
    if pd.isnull(text):
        return text
    
    # Limpieza básica
    text = str(text).strip()
    text = text.replace('""', '"').replace("''", "'")
    text = re.sub(r'\s+', ' ', text)
    text = remove_accents_and_special_chars(text)
    
    # Aplicar estandarización específica
    if field_type == 'phone':
        return standardize_phone_number(text)
    elif field_type == 'address':
        return standardize_address(text)
    elif field_type == 'name':
        # Para nombres: Primera letra mayúscula, resto minúscula
        return ' '.join(word.capitalize() for word in text.split())
    elif field_type == 'establishment':
        return text.upper()
    else:
        return text.upper()

def apply_standardization_to_dataframes(df_list: list[pd.DataFrame]) -> list[pd.DataFrame]:
    """
    Aplica estandarización de texto a una lista de DataFrames
    """
    standardized_dfs = []
    
    # Definir qué tipo de limpieza aplicar a cada columna
    field_mappings = {
        'TELEFONO': 'phone',
        'DIRECCION': 'address', 
        'DIRECTOR': 'name',
        'SUPERVISOR': 'name',
        'ESTABLECIMIENTO': 'establishment',
        'DEPARTAMENTO': 'general',
        'MUNICIPIO': 'general',
        'DISTRITO': 'general',
        'NIVEL': 'general',
        'SECTOR': 'general',
        'AREA': 'general',
        'STATUS': 'general',
        'MODALIDAD': 'general',
        'JORNADA': 'general',
        'PLAN': 'general',
        'DEPARTAMENTAL': 'general',
        'CODIGO': 'general'
    }
    
    print("Aplicando estandarización de texto...")
    
    for i, df in enumerate(tqdm(df_list, desc="Estandarizando DataFrames")):
        df_clean = df.copy()
        
        # Aplicar limpieza específica a cada columna
        for column, field_type in field_mappings.items():
            if column in df_clean.columns:
                df_clean[column] = df_clean[column].apply(
                    lambda x: comprehensive_text_cleaning(x, field_type)
                )
        
        standardized_dfs.append(df_clean)
    
    print(f"✅ Estandarización completada para {len(standardized_dfs)} DataFrames")
    return standardized_dfs

# Aplicar estandarización a los DataFrames que ya tienes
df_list_standardized = apply_standardization_to_dataframes(df_list)

print(f"\n📊 RESUMEN:")
print(f"• Total de archivos estandarizados: {len(df_list_standardized)}")
print(f"• Total de registros procesados: {sum(len(df) for df in df_list_standardized)}")

Aplicando estandarización de texto...


Estandarizando DataFrames: 100%|██████████| 85/85 [00:01<00:00, 44.18it/s]

✅ Estandarización completada para 85 DataFrames

📊 RESUMEN:
• Total de archivos estandarizados: 85
• Total de registros procesados: 38057


### Concatenar todo a un mismo .csv

In [11]:
# concatenate

### Estandarizar el csv conjunto

In [12]:
# standardize